<img src="data/photutils_banner.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

<div class="alert alert-block alert-info">
<h2 style="margin-top: 0">In this notebook, we will cover:</h2>

- The basics of PSF photometry using a simulated 2D Gaussian PSF
</div>

# Preliminaries

In [ ]:
# Initial imports
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.table import QTable
from astropy.visualization import simple_norm
from photutils.datasets import make_noise_image, make_test_psf_data
from photutils.detection import DAOStarFinder
from photutils.psf import IntegratedGaussianPRF, PSFPhotometry, IterativePSFPhotometry

# Change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.
%matplotlib inline

# Point Spread Function Photometry with Photutils

The Photutils PSF photometry module provides modular tools that allow users to completely customize the photometry procedure, e.g., by using different source detection algorithms, local background estimators, source groupers, and PSF models. Photutils provides implementations for each subtask involved in the photometry process, however, users are still able to include their own custom implementations.

This modularity is accomplished by using an object oriented programming approach that provides a more convenient user experience.

Photutils provides two basic classes to perform PSF photometry, `PSFPhotometry` and `IterativelyPSFPhotometry`. In this notebook, we will cover the basics of the `PSFPhotometry` class.

First, let's create a PSF model for our sources.

In [ ]:
psf_model = IntegratedGaussianPRF(flux=100, sigma=2.7 / 2.35)

## Create a simulated image

### with 250 stars in a 1000 x 1000 image

In [ ]:
psf_shape = (9, 9)
nsources = 250
shape = (1000, 1000)
data, true_params = make_test_psf_data(shape, psf_model, psf_shape, nsources,                    
                                       flux_range=(500, 1000), min_separation=25,
                                       seed=0, progress_bar=True)

Now let's add some noise to the image.

In [ ]:
noise = make_noise_image(data.shape, mean=0, stddev=0.1)
data += noise
error = np.abs(noise)
plt.figure(figsize=(5, 5))
norm = simple_norm(data, 'sqrt', min_cut=-0.1, max_cut=2.0)
plt.imshow(data, norm=norm);

The `true_params` output contains an Astropy table containing the true (x, y, flux) of our artificial sources.

In [ ]:
true_params

# The `PSFPhotometry` class

First, we create the `PSFPhotometry` class instance with a few parameters.

We must input a PSF model, which must be an Astropy `Fittable2DModel`.  Photutils provides several PSF models, including a [`GriddedPSFModel`](https://photutils.readthedocs.io/en/latest/api/photutils.psf.GriddedPSFModel.html#photutils.psf.GriddedPSFModel) for spatially-varying PSFs.

We must also input the `fit_shape` parameter, which defines the region around the center of the PSF that is used for fitting the model.

The `finder` keyword takes a Photutils finder object.  Here, we use the `DAOStarFinder`.  We also input an `aperture_radius` (in pixel) to estimate the initial source fluxes.

In [ ]:
fit_shape = (5, 5)
fit_shape = 5
finder= DAOStarFinder(6.0, 2.0)
psfphot = PSFPhotometry(psf_model, fit_shape, finder=finder, aperture_radius=5, progress_bar=True)

To perform the PSF fitting, we call the `psfphot` object on the data and optional error array.  The result is an Astropy Table. Here, we print only the first 10 rows.

In [ ]:
phot = psfphot(data, error=error)
phot[:10]

We can use the `make_model_image` method to create a PSF model image of the results.

In [ ]:
model_img = psfphot.make_model_image(data.shape, psf_shape)
plt.imshow(model_img, norm=norm);
plt.colorbar()

We can use the `make_residual_image` method to create a residual image.

In [ ]:
resid = psfphot.make_residual_image(data, psf_shape)
plt.imshow(resid, norm=norm);
plt.colorbar()

Our residual image is just noise without any sources, which indicates excellent PSF model fits.

## Inputing the initial parameters

Instead of using a star finder, we can pass in a Table of the initial (x, y) positions.
This table can also include the initial fluxes.

Let's use the `true_params` table, but shift the (x, y, flux) values from their true values.

In [ ]:
rng = np.random.default_rng(seed=123)
init_params = true_params.copy()
init_params['x'] = init_params['x'].astype(int) + rng.uniform(-0.5, 0.5, 250)
init_params['y'] = init_params['y'].astype(int) + rng.uniform(-0.5, 0.5, 250)
init_params['flux'] = init_params['flux'].astype(int) + rng.uniform(-10, 10, 250)
init_params[:10]

Since we are inputing the initial (x, y) values, we no longer need to input the `finder`.  Likewise, since we are inputing the initial flux values, we no longer need to input the `aperture_radius`.

In [ ]:
psfphot2 = PSFPhotometry(psf_model, fit_shape, progress_bar=True)
phot2 = psfphot2(data, error=error, init_params=init_params)
phot2[:10]

In [ ]:
resid2 = psfphot2.make_residual_image(data, psf_shape)
plt.imshow(resid2, norm=norm);
plt.colorbar()

Since the sources are ordered, we can directly compare the fit values with the true values.

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(true_params['x'], phot2['x_fit'], '.')
plt.xlabel('x True')
plt.ylabel('x Fit');

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(true_params['y'], phot2['y_fit'], '.')
plt.xlabel('y True')
plt.ylabel('y Fit');

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(true_params['flux'], phot2['flux_fit'], '.')
plt.xlabel('Flux True')
plt.ylabel('Flux Fit');

In [ ]:
pdiff = (true_params['flux'] - phot2['flux_fit']) / true_params['flux'] * 100.0
np.max(np.abs(pdiff))

In [ ]:
plt.hist(pdiff, bins=50)
plt.xlabel('True/Fit Flux Percent Difference');

Please consult the [PSF Photometry documentation](https://photutils.readthedocs.io/en/1.10.0/psf.html) for additional features including:

- Fitting a single (or few) sources in an image
- Forced Photometry
- Source Grouping
- Local Background Subtraction
- Iterative PSF Photometry